### Background and history
*COMING SOON*

### Justification
*COMING SOON*

### Objectives
*COMING SOON*

### Workflow

*COMING SOON*

### Suggested reading (prior to running the tutorial)

- [A mathematical review of the Fourier transform - SEG Wiki](https://wiki.seg.org/wiki/A_mathematical_review_of_the_Fourier_transform)
- [An Intuitive Explanation of Fourier Theory](https://www.semanticscholar.org/paper/An-Intuitive-Explanation-of-Fourier-Theory-Lehar/3d67a901be76a2258db924799bd88e258032bb6b)
- [The 1-D Fourier transform - SEG Wiki](https://wiki.seg.org/wiki/The_1-D_Fourier_transform)
- [The 2-D Fourier transform - SEG Wiki](https://wiki.seg.org/wiki/The_2-D_Fourier_transform)
- [Elegant SciPy - Chapter 4: Frequency and the Fast Fourier Transform](https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch04.html)

### Import all necessary packages

In [1]:
import numpy as np
import scipy as sp
from scipy import ndimage as ndi
from scipy import signal
from numpy.fft import fft2, ifft2, fftshift, ifftshift
from kneed import KneeLocator
import skimage.morphology as morph
from skimage import io, exposure
from skimage.util import img_as_float
from skimage.filters import gaussian
from skimage.filters.rank import entropy
from skimage.color import rgb2gray
from skimage.feature import peak_local_max, canny
from skimage.restoration import denoise_tv_chambolle
from skimage.draw import  circle, circle_perimeter, ellipse, rectangle

from skimage.measure import compare_ssim, find_contours
import matplotlib.pyplot as plt
import matplotlib.colors as clr

### Load test data: import King Tut CT scan image

Source: Supreme Council of Antiquities,  guardians.net/hawass/press_release_tutankhamun_ct_scan_results.htm

##### Function to normalize arrays to [0 1] range

In [2]:
def normalise(arr):
    return (arr-np.amin(arr))/(np.amax(arr)-np.amin(arr))

In [4]:
data = normalise(rgb2gray(io.imread('images_and_data/Tut.jpg')))
print(np.shape(data), np.amin(data), np.amax(data))

FileNotFoundError: No such file: '/Users/matteoniccoli/Pythonwork/GitHub/t21-hack-footprint/dev/images_and_data/Tut.jpg'

##### Add some blur

In [ ]:
data=gaussian(data, 1)

In [ ]:
plt.figure(figsize=(15,12))
plt.imshow(data, cmap='bone'), plt.axis('off'); 

##### All-in-one function to perform Fourier transform, get magnitude, center on low frequency (for display purposes)

In [ ]:
def numpy_fft2(im):
    F = np.fft.fft2(im)     # Perform 2-dimensional discrete Fourier transform 
    C = np.fft.fftshift(F)  # Center spectrum on minimum frequency
    Mag = np.abs(C).real    # Get magnitude and phase
    Ph  = np.angle(C).real    
    return Mag, Ph

In [ ]:
A, _ = numpy_fft2(data)

In [ ]:
Ag = denoise_tv_chambolle(np.log(A +1), weight=0.2)  
lg = np.log(A +1)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(lg,  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

#### Get azimuthally averaged radial profile

To show that the data "lives" in the lowest frequencies.

Radial mean code adapted from [SciPy lectures](http://scipy-lectures.org/advanced/image_processing/#measuring-objects-properties-ndimage-measurements) (his method, compared to other ones tested, works better in that I can set the number of bins to be same as the size of the input image).

Automatic knee-point detection with the [Kneedle algorithm](https://raghavan.usc.edu//papers/kneedle-simplex11.pdf) using the [Kneed package](https://github.com/arvkevi/kneed).

__NB. It may be a bit better to use second derivative. Implement later (not high priority.__

In [ ]:
sx, sy = Ag.shape
X, Y = np.ogrid[0:sx, 0:sy]
r = np.hypot(X - sx/2, Y - sy/2)
rbin = (502* r/r.max()).astype(np.int)
with np.errstate(invalid='ignore'):
    radial_mean = ndi.mean(Ag**2, labels=rbin, index=np.arange(1, rbin.max() +1))
radial_mean = radial_mean[~np.isnan(radial_mean)]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
plt.plot(radial_mean, 'b')
ax.set_xlabel('Pixels', fontsize=14)
ax.set_ylabel('Power spectrum', fontsize=14)
ax.grid(which='both')

In [ ]:
kl = KneeLocator(np.arange(0,len(radial_mean)), radial_mean, curve='convex', direction='decreasing',  online=True)
print(kl.knee)

In [ ]:
kl.plot_knee_normalized()
fig = plt.gcf()
fig.set_size_inches(12,12)
#plt.gca().invert_yaxis()
plt.grid(which='both')

In [ ]:
radius = int(kl.knee/2)
rr, cc = circle_perimeter(np.shape(Ag)[0]//2, np.shape(Ag)[0]//2, radius)
lg[rr, cc] = np.amax(lg)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(lg,  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

### Preliminary examples of 2D Fourier Transform of straight lines and sinusoids

##### How does the Fourier spectrum of a vertical line looks like?

Reference blog post from Steve Eddins (Mathworks): ["Fourier transforms, vertical lines, and horizontal lines"](https://blogs.mathworks.com/steve/2010/09/22/fourier-transforms-vertical-lines-and-horizontal-lines/)

In [ ]:
vline = np.zeros((201,201))
vline[:,100]=1
plt.figure(figsize=(12,12))
plt.imshow(vline, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
A, _ = numpy_fft2(vline)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(np.log(A +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

##### Next, how does the Fourier spectrum of a 2D vertical cosine wave look like?

##### Sample rate

In [ ]:
f = 0.02

In [ ]:
x = y = np.arange(400)  
xx, yy = np.meshgrid(x, y)

In [ ]:
vcos=normalise(np.cos(np.pi*15*f*xx))

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(vcos, cmap='gray', interpolation = 'none'), plt.axis('off'); 

In [ ]:
A, _ = numpy_fft2(vcos)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(np.log(A +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

In the image above, the center dot is the DC point. The frequency of the cosine wave is represented by the distance of either of the other points to the DC point. The amplitude information is encoded in the intensity of the points.

##### One more thing. 

Since ultimately we want to be able to filter a signal by manipulating the spectrum, then inverse Fourier transform the signal, let's show that (as should be expected) there is no signal loss by taking an image through fft2 and back via ifft2.

##### Method 1, the `numpy` way
     
That is, `ifft2(fft2(arr)) == arr` [to within a reasonable numerical accuracy](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.fft.ifft2.html) 

In [ ]:
accuracy = 1e-12

In [ ]:
print(np.allclose(vcos, ifft2(fft2(vcos)), accuracy, accuracy))

print(np.allclose(ifft2(fft2(vcos)), vcos, accuracy, accuracy))

##### Method 2, the `scikit-image` way

Reference: [Structural similarity index](https://scikit-image.org/docs/0.12.x/auto_examples/transform/plot_ssim.html)

In [ ]:
def mse(x, y):
    return np.linalg.norm(x - y)

In [ ]:
diff_mse = mse(vcos, ifft2(fft2(vcos)).real)

In [ ]:
diff_ssim = compare_ssim(vcos, ifft2(fft2(vcos)).real)

In [ ]:
diff = img_as_float(vcos - ifft2(fft2(vcos)).real)

In [ ]:
print ('MSE: %.2E, SSIM: %.2f' % (diff_mse, diff_ssim))

### Create synthetic acquisition footprint

We will create a 2D acquisition footprint by combining two non-orthogonal sinusoids of different frequency.

##### First cosine wave

In [ ]:
cswA=normalise(np.cos(-2*np.pi*0.5*f*xx+6*np.pi*0.5*f*yy))

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(cswA, cmap='gray', interpolation = 'none'), plt.axis('off'); 

In [ ]:
A, _ = numpy_fft2(cswA)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(np.log(A +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

##### Second cosine wave

In [ ]:
cswB=normalise(np.cos(-15*0.6*np.pi*f*xx+1*0.6*np.pi*f*yy))
print(np.shape(cswB))

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(cswB, cmap='gray', interpolation = 'none'), plt.axis('off'); 

##### Combine the two cosine waves

In [ ]:
temp = cswA + cswB

In [ ]:
footprint = normalise(temp)
np.shape(footprint)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(footprint, cmap='gray', interpolation = 'none'), plt.axis('off'); 

##### Raised cosine function to taper the input _à la_ Steve Eddins, to help with DFT periodicity i artifacts

Reference blog post: [_"Fourier transform visualization using windowing"_](https://blogs.mathworks.com/steve/2009/12/04/fourier-transform-visualization-using-windowing)

In [ ]:
def raised_cosine(im):
    m, n = np.shape(im)
    w1 = np.cos(np.linspace(-np.pi/2, np.pi/2, m))
    w1 = w1[:, None] 
    w2 = np.cos(np.linspace(-np.pi/2, np.pi/2, n))
    w = w1*w2
    return normalise(im * w)

In [ ]:
f = raised_cosine(footprint)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(f, cmap='gray', interpolation = 'none'), plt.axis('off'); 

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(f, cmap='gray', interpolation = 'none'), plt.axis('off'); 

In [ ]:
print(np.amin(f), np.amax(f))

In [ ]:
A, _ = numpy_fft2(f)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(np.log(A +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

##### For comparison, this is the result if we do not use the tapering to remove artifacts

In [ ]:
Au, _ = numpy_fft2(footprint)
plt.figure(figsize=(12, 12))
plt.imshow(np.log(Au +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

Where do those vertical lines come from? SImilarly to what Steve showed in the blog post, now that we tiled the signal (simulating the Fourier transform's implicit periodicity) a horizontal edge effect becomes apparent.

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(np.tile(footprint, (2,2)),  cmap='gray', interpolation = 'none'), plt.axis('off');

### Combine data to synthetic acquisition footprint

In [ ]:
noisy = normalise(data.copy() + f*4)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(noisy, cmap='bone'), plt.axis('off');

In [ ]:
fkt = raised_cosine(noisy)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(fkt, cmap='bone', interpolation = 'none'), plt.axis('off'); 

#### Check the spectrum

In [ ]:
A, _ = numpy_fft2(fkt)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(np.log(A +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

In [ ]:
Ag = denoise_tv_chambolle(np.log(A +1), weight=0.2)  

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Ag,  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

### Filter out the footprint


##### Find local maxima in Fourier Transform image using Scikit-image 

https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_peak_local_max.html

In [ ]:
fp = morph.disk(5)

In [ ]:
image_max = ndi.maximum_filter(Ag, footprint = fp, mode='constant')
coordinates= peak_local_max(Ag, min_distance=5, num_peaks=5)
coordinates_center = peak_local_max(Ag, min_distance=10, num_peaks=1)

In [ ]:
# display results
fig, axes = plt.subplots(2, 2, figsize=(24, 24), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(Ag, cmap='gray')
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(image_max, cmap='gray')
ax[1].axis('off')
ax[1].set_title('Maximum filter')

ax[2].imshow(Ag, cmap='gray')
ax[2].autoscale(False)
ax[2].plot(coordinates_center[:, 1], coordinates_center[:, 0], 'mo')
ax[2].axis('off')
ax[2].set_title('Peak 1 local maximum')

ax[3].imshow(Ag, cmap='gray')
ax[3].autoscale(False)
ax[3].plot(coordinates[:, 1], coordinates[:, 0], 'mo')
ax[3].axis('off')
ax[3].set_title('Peak 5 local maxima')

fig.tight_layout()

plt.show()


##### Maximum filter also returns an image directly! Let's take advantage of that:

In [ ]:
fttt2_im = peak_local_max(Ag, min_distance=10, num_peaks=5, indices = False)*1

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(fttt2_im,  
           cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
fttt2_im_center = peak_local_max(Ag, min_distance=10, num_peaks=1, indices = False)*1
fttt2_im_no_center = fttt2_im - fttt2_im_center 

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(fttt2_im_no_center,  
           cmap='gray', interpolation = 'none'), plt.axis('off');

##### Design footprint suppression filter by convolution with Gaussian disk 

Initially used `convolve` and `Gaussian2DKernel` [from Astropy](https://docs.astropy.org/en/stable/convolution/kernels.html) but then moved to `Scipy` for both, with significantly improved performance (you can read about it [here](https://github.com/mycarta/2D_FFT_filter_tutorial/blob/master/speed_up_convolution.ipynb) and [here](https://github.com/mycarta/2D_FFT_filter_tutorial/blob/master/scipy_gaussian_kernel.ipynb)).

In [ ]:
def scipy_gaussian_2D(std):
    '''
    2D Gaussian filter kernel similar to astropy\'s Gaussian2DKernel
    (https://docs.astropy.org/en/stable/api/astropy.convolution.Gaussian2DKernel.html#astropy.convolution.Gaussian2DKernel)
    using scipy.signal.gaussian 
    (and inspired by https://gist.github.com/thomasaarholt/267ec4fff40ca9dff1106490ea3b7567)
    
    Parameters: 
    std (int) : standard deviation of the Gaussian in pixels
    
    Returns:
    out (2D array): 2D Gaussian filter kernel
    '''
    ksp1D = signal.gaussian(std*8+1, std)
    ksp2D = np.outer(ksp1D, ksp1D)
    ksp2D /= (2*np.pi*(std**2))
    return ksp2D

In [ ]:
gauss_kernel = scipy_gaussian_2D(2)
filter0 = signal.fftconvolve(fttt2_im_no_center, gauss_kernel,  mode='same')

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(filter0, cmap='cubehelix', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(1-filter0, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
print(np.amin(filter0), np.amax(filter0))

In [ ]:
filter0=normalise(filter0)

##### Show why a gaussian kernel filter is better than a disk notch filter

In [ ]:
c1 = np.zeros((500, 500, 3), 'float')
rr, cc = circle(250, 250, 15, c1.shape)
c1[rr,cc] = (1.)
c1=rgb2gray(c1)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(c1, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(fftshift(ifft2(ifftshift(c1))).real, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
c2 = signal.fftconvolve(c1, scipy_gaussian_2D(7),  mode='same')

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(c2, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(fftshift(ifft2(ifftshift(c2))).real, cmap='gray', interpolation = 'none'), plt.axis('off');

##### Apply filter and check reconstructed cat scan image

In [ ]:
filtered = ifft2(np.multiply(ifftshift(1-filter0), fft2(data.copy()))).real

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(filtered, cmap='gray', interpolation = 'none'), plt.axis('off');

##### Check effect of filtering on Canny edge effectiveness

In [ ]:
def auto_canny(img, sigma = 0.33):
    """Zero-parameter, automatic Canny edge detection using scikit-image.
    Original function from pyimagesearch: Zero-parameter, automatic Canny edge with with Python and OpenCV
    www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv"""
        
    # compute the median of the single channel pixel intensities
    v = np.median(img)
 
    # apply automatic Canny edge detection using the computed median
    lower = float(max(0.0, (1.0 - sigma) * v))
    upper = float(min(1.0, (1.0 + sigma) * v))
    edged = canny(img, sigma, lower, upper)

    # return the edged image
    return edged

In [ ]:
Cn = auto_canny(noisy)

In [ ]:
Cdn = auto_canny(filtered)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Cn, cmap='gray_r', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Cdn, cmap='gray_r', interpolation = 'none'), plt.axis('off');

##### Check effect of filtering on sobel filters effectiveness (1D and 2D)

In [ ]:
Snh = ndi.sobel(noisy,0)
Sdnh = ndi.sobel(filtered,0)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Snh, cmap='gray_r', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Sdnh, cmap='gray_r', interpolation = 'none'), plt.axis('off');

In [ ]:
Snv = ndi.sobel(noisy,1)
Sdnv = ndi.sobel(filtered,1)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Snv, cmap='gray_r', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Sdnv, cmap='gray_r', interpolation = 'none'), plt.axis('off');

In [ ]:
def Sobel_2d(data):
    '''2D Sobel filter'''
    dx = ndi.sobel(data, 0)  # horizontal derivative
    dy = ndi.sobel(data, 1)  # vertical derivative
    mag = np.hypot(dx, dy)      # magnitude
    mag *= 255.0 / np.max(mag)  # normalize
    return mag

In [ ]:
Sn2 = Sobel_2d(noisy)

In [ ]:
Sdn2 = Sobel_2d(filtered)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Sn2, cmap='gray_r', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Sdn2, cmap='gray_r', interpolation = 'none'), plt.axis('off');

__Add discussion on 2D sobel filter__

### Some real seismic data now

##### Function to pad input to a square NxN array. Not necessary in this case because images are square to begin with.

In [ ]:
def pad_next_square_size(im):
    """Function to pad a rectangualr image to a square image.
    Parameters:
    im (2D array): input grayscale image
    
    Returns:
    out (2D array): padded input image
    padding (slice object): a slice object that can be used later on to unpad
    
    Example:
    out, padding = pad_next_square_size(im)
    """
    m, n = np.shape(im) # get input shape
    deficit = max([m,n])- min([m,n])  # get deficit between size lengths

    # difference in dimensions is even, pad both sides of short dimension by deficit//2
    if deficit%2 == 0: 
        deficit1 = deficit//2
        deficit2 = deficit//2
    
    # difference in dimensions is odd, pad one side by deficit//2 +1
    else:
        deficit1 = deficit//2
        deficit2 = deficit1+1
        
    if m > n:
        print("Padded image columns")
        return (np.pad(im,((0,0), (deficit1,deficit2)), 'reflect'), slice(deficit1, -deficit2))
        
    else:
        print("Padded image rows")
        return (np.pad(im,((deficit1,deficit2), (0,0)), 'reflect'), slice(deficit1, -deficit2))

In [ ]:
sc = normalise(rgb2gray(io.imread('images and data/Luo_edge-preserving_smoothing.png')))
plt.figure(figsize=(12, 12))
plt.imshow(sc, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
print (np.shape(sc))
print(np.amax(sc))

In [ ]:
test1, slc = pad_next_square_size(sc)

In [ ]:
slc

In [ ]:
A, _ = numpy_fft2(raised_cosine(test1))
plt.figure(figsize=(12, 12))
plt.imshow(np.log(A +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(test1, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
A, _ = numpy_fft2(raised_cosine(test1))
plt.figure(figsize=(12, 12))
plt.imshow(np.log(A +1),  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

In [ ]:
Ag = denoise_tv_chambolle(np.log(A +1), weight=0.2)  

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Ag,  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

In [ ]:
image_max = ndi.maximum_filter(Ag, footprint = fp, mode='constant')
coordinates= peak_local_max(Ag, min_distance=10, num_peaks=5)
coordinates_center = peak_local_max(Ag, min_distance=10, num_peaks=1)

In [ ]:
# display results
fig, axes = plt.subplots(2, 2, figsize=(24, 24), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(Ag, cmap='gray')
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(image_max, cmap='gray')
ax[1].axis('off')
ax[1].set_title('Maximum filter')

ax[2].imshow(Ag, cmap='gray')
ax[2].autoscale(False)
ax[2].plot(coordinates_center[:, 1], coordinates_center[:, 0], 'mo')
ax[2].axis('off')
ax[2].set_title('Peak 1 local maximum')

ax[3].imshow(Ag, cmap='gray')
ax[3].autoscale(False)
ax[3].plot(coordinates[:, 1], coordinates[:, 0], 'mo')
ax[3].axis('off')
ax[3].set_title('Peak 5 local maxima')

fig.tight_layout()
plt.show()

In [ ]:
fttt2_im = peak_local_max(Ag, min_distance=10, num_peaks=5, indices = False)*1

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(fttt2_im,  
           cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
fttt2_im_center = peak_local_max(Ag, min_distance=10, num_peaks=1, indices = False)*1
fttt2_im_no_center = fttt2_im - fttt2_im_center 

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(fttt2_im_no_center,  
           cmap='gray', interpolation = 'none'), plt.axis('off');

##### Design footprint suppression filter by convolution with Gaussian disk 

Using `astropy`:
https://docs.astropy.org/en/stable/convolution/kernels.html

In [ ]:
gauss_kernel1 = scipy_gaussian_2D(5)
filter1 = signal.fftconvolve(fttt2_im_no_center, gauss_kernel,  mode='same')

print (np.amin(filter1))
print (np.amax(filter1))

filter1=normalise(filter1)
print (np.amin(filter1))
print (np.amax(filter1))

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(filter1, cmap='cubehelix', interpolation = 'none'), plt.axis('off');

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(1-filter1, cmap='gray', interpolation = 'none'), plt.axis('off');

##### Apply filter and view reconstructed image

In [ ]:
def reverse_padding(im, filtered_im, slc):
    m, n = np.shape(im) # get input shape
     
    if m > n:
        print("Unpadding image columns")
        return filtered_im[:, slc]       
    else:
        print("Unpadding image rows")
        return filtered_im[slc, :]      

In [ ]:
filtered1 = ifft2(np.multiply(ifftshift(1-filter1), fft2(test1.copy()))).real
print(np.shape(filtered1))

filtered1 = reverse_padding(sc, filtered1, slc)
print(np.shape(filtered1))

In [ ]:
compare_ssim(sc, filtered1)

In [ ]:
diff = img_as_float(sc - filtered1)

In [ ]:
# display results
fig, axes = plt.subplots(1, 3, figsize=(18, 12), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(sc, cmap='gray')
ax[0].axis('off')
ax[0].set_title('Original')
ax[1].imshow(filtered1, cmap='gray')
ax[1].axis('off')
ax[1].set_title('Filtered')
ax[2].imshow(diff, cmap='gray')
ax[2].axis('off')
ax[2].set_title('Difference')
fig.tight_layout()
plt.show()

### One more photo, for fun

##### A Scene in Shantytown, March 1880

This is the first ever newspaper photo printed using a halftone screen!

Source: http://www.sfu.ca/humanities-institute/contours/issue8/theory/5.html

Image file: http://www.sfu.ca/content/dam/sfu/humanities-institute/Images/contours/issue8/FIG%2010.2_A-SCENE-IN-SHANTYTOWN.jpg

History: https://en.wikipedia.org/wiki/The_Daily_Graphic#History

In [ ]:
data = normalise(rgb2gray(io.imread('images and data/Scene_Shantytown_Horgan.jpg')))
print(np.shape(data), np.amin(data), np.amax(data))

In [ ]:
plt.figure(figsize=(15,12))
plt.imshow(data, cmap='bone'), plt.axis('off'); 

In [ ]:
sc = normalise(data)

In [ ]:
test1, slc = pad_next_square_size(sc)

In [ ]:
A, _ = numpy_fft2(raised_cosine(test1))

In [ ]:
Ag = denoise_tv_chambolle(np.log(A +1), weight=0.2)  

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(Ag,  cmap='cubehelix', interpolation = 'none'), plt.axis('off');

#### Stacking rows to get those maxima

In [ ]:
stack_rows = np.sum(Ag, axis=0)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(stack_rows, 'r')
plt.yticks([])
plt.show()

In [ ]:
multiplicity = 2 # number of expected peaks except for central

indexes, _ = signal.find_peaks(stack_rows, distance = len(stack_rows)/(2*multiplicity))
indexes = np.delete(indexes, len(indexes)//2)

In [ ]:
test = np.zeros(np.shape(Ag))
rr, cc = ellipse(np.shape(Ag)[0]//2, indexes[0], 21, 21)
rr1, cc1 = ellipse(np.shape(Ag)[0]//2, indexes[-1], 21, 21)
test[rr, cc] = 1
test[rr1, cc1] = 1

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(test,  
           cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
gauss_kernel = scipy_gaussian_2D(15)
filter00 = signal.fftconvolve(test, gauss_kernel, mode='same')
filter00 = normalise(filter00)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(1-filter00, cmap='gray', interpolation = 'none'), plt.axis('off');

In [ ]:
filtered1 = ifft2(np.multiply(ifftshift(1-filter00), fft2(test1.copy()))).real
print(np.shape(filtered1))

filtered1 = reverse_padding(sc, filtered1, slc)
print(np.shape(filtered1))

In [ ]:
compare_ssim(sc, filtered1)

In [ ]:
diff = img_as_float(sc - filtered1)

In [ ]:
# display results
fig, axes = plt.subplots(3, 1, figsize=(16, 24), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(sc, cmap='gray')
ax[0].axis('off')
ax[0].set_title('Original')
ax[1].imshow(filtered1, cmap='gray')
ax[1].axis('off')
ax[1].set_title('Filtered')
ax[2].imshow(diff, cmap='gray')
ax[2].axis('off')
ax[2].set_title('Difference')
fig.tight_layout()
plt.show()

### A complete seismic example

Let's use some time slices from the [Penobscot 3D](https://terranubis.com/datainfo/Penobscot), which is available under a [CC BY-SA license](https://creativecommons.org/licenses/by-sa/3.0/).

#### Import the seismic data previously saved to npy file

#### Access and display time slice at 948 ms which is at about the median time of Horizon B

In [ ]:
penobscot = np.load('images and data/penobscot.npy')

In [ ]:
samples = np.load('images and data/samples.npy')
samples

In [ ]:
slc948 = np.where(samples==948)[0]
print(slc948)

In [ ]:
t948 = normalise(np.squeeze(penobscot[:,:,slc948]))
np.shape(t948)

In [ ]:
fig = plt.figure(figsize=(22, 9))
ax = fig.add_subplot(1, 1, 1)
ax.set_xticks([])
ax.set_yticks([])
plt.imshow(t948, cmap='gray_r', aspect = 0.5, origin = 'lower', interpolation = 'none');  
# aspect = 0.5 since the Inline bin size is 12.5 m/line and the Crossline bin size is 25 m/line
plt.colorbar();

#### View spectrum

In [ ]:
test1, slc = pad_next_square_size(t948)

In [ ]:
slc

In [ ]:
A, _ = numpy_fft2(raised_cosine(test1))
Ag = denoise_tv_chambolle(np.log(A +1), weight=0.2)  

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1)
plt.imshow(np.log(A +1),  cmap='cubehelix', origin = 'lower', interpolation = 'none'), plt.axis('off');

#### Design filter
This time using rectangles; another good example of a good interactive feature to have (user drawn rectangles)

In [ ]:
rec= np.zeros(np.shape(A), dtype=np.uint8)
start1 = (np.shape(Ag)[0]//2+5,0)
end1 = (np.shape(Ag)[0]//2-5,np.shape(Ag)[0]//2-50)
rr1, cc1 = rectangle(start1, end=end1, shape=test.shape)
rec[rr1, cc1] = 1

start2 = (np.shape(Ag)[0]//2+5,np.shape(Ag)[0]//2+50)
end2 = (np.shape(Ag)[0]//2-5,np.shape(Ag)[0]-1)
rr2, cc2 = rectangle(start2, end=end2, shape=test.shape)
rec[rr2, cc2] = 1

#### Show filter design mask on top of spectrum

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1)
plt.imshow(np.log(A +1),  cmap='cubehelix', origin = 'lower', interpolation = 'none'), plt.axis('off');
plt.imshow(rec,  cmap='gray', origin = 'lower', interpolation = 'none', alpha = 0.2), plt.axis('off');

#### Apply gaussian taper and display final filter on top of spectum

In [ ]:
gauss_kernel = scipy_gaussian_2D(11)
filter00 = signal.fftconvolve(rec, gauss_kernel, mode='same')
filter00 = normalise(filter00)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1)
plt.imshow(np.log(A +1),  cmap='cubehelix', origin = 'lower', interpolation = 'none'), plt.axis('off');
plt.imshow(1-filter00, cmap='gray', origin = 'lower', interpolation = 'none', alpha = 0.2), plt.axis('off');

#### Apply filter and show input, output, difference

In [ ]:
filtered1 = ifft2(np.multiply(ifftshift(1-filter00), fft2(test1.copy()))).real
print(np.shape(filtered1))

filtered1 = reverse_padding(t948, filtered1, slc)
print(np.shape(filtered1))

In [ ]:
diff = img_as_float(t948 - filtered1)

In [ ]:
# display results
fig, axes = plt.subplots(3, 1, figsize=(16, 26), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(t948, cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered1, cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[1].axis('off')
ax[1].set_title('Filtered')

ax[2].imshow(diff, cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[2].axis('off')
ax[2].set_title('Difference')

fig.tight_layout()
plt.show()

#### Now we test the same filter on a deeper time slice, to ensure it is not too aggressive
The time slice at 1136 ms is at about the median time of Horizon C

In [ ]:
slc1136 = np.where(samples==1136)[0]
print(slc1136)

In [ ]:
t1136 = normalise(np.squeeze(penobscot[:,:,slc1136]))

In [ ]:
test1, slc = pad_next_square_size(t1136)

#### Apply filter and show input, output, difference

In [ ]:
filtered1 = ifft2(np.multiply(ifftshift(1-filter00), fft2(test1.copy()))).real
print(np.shape(filtered1))

filtered1 = reverse_padding(t1136, filtered1, slc)
print(np.shape(filtered1))

In [ ]:
diff = img_as_float(t1136 - filtered1)

In [ ]:
# display results
fig, axes = plt.subplots(3, 1, figsize=(16, 26), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(t1136, cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered1, cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[1].axis('off')
ax[1].set_title('Filtered')

ax[2].imshow(diff, cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[2].axis('off')
ax[2].set_title('Difference')

fig.tight_layout()
plt.show()

#### Apply filter to the whole cube then calculate semblance (before and after) and compare results to validate method

The semblance code below is from [Semblance, coherence, and other discontinuity attributes
](https://github.com/seg/tutorials-2015/blob/master/1512_Semblance_coherence_and_discontinuity/Discontinuity_tutorial.ipynb) a fantastic Geophysical tutorial bt Joe Kington, published in The Leading Edge in December 2015.

Running the cell below will take a looong time to run, please lad the .npy file instead.

In [ ]:
#def moving_window(data, window, func):
#    # `generic_filter` will give the function 1D input. We'll reshape it for convinence
#    wrapped = lambda region: func(region.reshape(window))
    
#    # Instead of an explicit for loop, we'll use a scipy function to do the same thing
#    # The boundaries will be handled by "reflecting" the data, by default
#    return sp.ndimage.generic_filter(data, wrapped, window)

In [ ]:
#def gersztenkorn_eigenstructure(region):
#    # Once again, stack all of the traces into one 2D array.
#    region = region.reshape(-1, region.shape[-1])

#    cov = region.dot(region.T)
#    vals = np.linalg.eigvalsh(cov)
#    return vals.max() / vals.sum()

In [ ]:
#gersztenkorn = moving_window(penobscot, (3, 3, 9), gersztenkorn_eigenstructure)

In [ ]:
#np.save('images and data/gersztenkorn.npy', gersztenkorn)

In [ ]:
#gersztenkorn = np.load('images and data/gersztenkorn.npy')

__N.B. The cell below is not efficient. It is the code to apply the 2D filter to the full 3D volume.
It needs to be rewritten using broadcasting.__

In [ ]:
#penobscot_filt = penobscot.copy()
#for i,_ in enumerate(p.samples):
#    ts, slc = pad_next_square_size(penobscot_filt[:,:,i])
#    temp = ifft2(np.multiply(ifftshift(1-filter00), fft2(ts.copy()))).real
#    penobscot_filt[:,:,i] = reverse_padding(penobscot[:,:,i], temp, slc)

In [ ]:
#np.save('images and data/penobscot_filtered.npy', penobscot_filt)

In [ ]:
#penobscot_filt = np.load('images and data/penobscot_filtered.npy')

In [ ]:
#gersztenkorn_filt = moving_window(penobscot_filt, (3, 3, 9), gersztenkorn_eigenstructure)

In [ ]:
#np.save('images and data/gersztenkorn_filtered.npy', gersztenkorn_filt)

In [ ]:
gersztenkorn = np.load('images and data/gersztenkorn.npy')
gersztenkorn_filt=np.load('images and data/gersztenkorn_filtered.npy')

In [ ]:
# display results
fig, axes = plt.subplots(2, 1, figsize=(14, 16), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(np.squeeze(normalise(gersztenkorn[:,:,slc1136])), cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[0].axis('off')
ax[0].set_title('Semblance of original data')

ax[1].imshow(np.squeeze(normalise(gersztenkorn_filt[:,:,slc1136])), cmap='gray', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[1].axis('off')
ax[1].set_title('Semblance of filtered data')

fig.tight_layout()
plt.show()

In [ ]:
# display results
fig, axes = plt.subplots(2, 1, figsize=(14, 16), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(normalise(Sobel_2d((np.squeeze(gersztenkorn[:,:,slc1136])))), cmap='gray_r', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[0].axis('off')
ax[0].set_title('2D Sobel result on the semblance of original data')   

ax[1].imshow(normalise(Sobel_2d(np.squeeze(gersztenkorn_filt[:,:,slc1136]))), cmap='gray_r', aspect = 0.5, origin = 'lower', interpolation = 'none')
ax[1].axis('off')
ax[1].set_title('2D Sobel result on the semblance of filtered data')

fig.tight_layout()
plt.show()

### ANOTHER IMAGE EXAMPLE

__Moon landing picture__

*COMING SOON*

### A FORENSIC EXAMPLE

__Try fingerprint clean-up using image from [this example](https://4n6site.com/improc/fftplugin/howto.htm)__

*COMING SOON*